In [ ]:
import boto3

kinesis = boto3.client('kinesis')
iam = boto3.client('iam')

In [ ]:
import datetime
import json
import random
import uuid
import time

class PositionWriter:
    def __init__(self, stream_name, kinesis_client):
        self.stream_name = stream_name
        self.kinesis_client = kinesis_client
        
    def write_position(self, owner, symbol, amount):
        data = {
            "owner" : owner,
            "symbol" : symbol,
            "amount" : amount
        }
        
        print(data)
        
        self.kinesis_client.put_record(
            StreamName=self.stream_name,
            Data=json.dumps(data),
            PartitionKey=owner)
        
class QuoteWriter:
    def __init__(self, stream_name, kinesis_client):
        self.stream_name = stream_name
        self.kinesis_client = kinesis_client
        
    def write_quote(self, symbol, price):
        data = {
            'TICKER': symbol,
            'PRICE': price
        }
        
        print(data)
        
        # not needed - used for compatibility with copy/paste stream schema
        cloud_event = {
            "specversion" : "1.0",
            "type" : "tick",
            "source" : "sample-stream",
            "subject" : "delayed-data",
            "id" : str(uuid.uuid4()),
            "time" : datetime.datetime.now().isoformat(),
            "datacontenttype" : "application/json",
            "data" : data
        }
        
        self.kinesis_client.put_record(
            StreamName=self.stream_name,
            Data=json.dumps(cloud_event),
            PartitionKey=symbol # TODO - determine decent parition key given real quote data distribution
        )
        
        

In [ ]:
# Quote write uses 'AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV' for symbols

In [ ]:
positionWriter = PositionWriter("QuoteStream", kinesis)

In [ ]:
positionWriter.write_position("x001","AMZN", 10)

In [ ]:
qw = QuoteWriter("QuoteStream", kinesis)

In [ ]:
qw.write_quote('AMZN', 10)

In [ ]:
qw.write_quote('AMZN', 100)

Stream reader output should contain...

```
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":10.0,"MKT_VALUE":100.0}'
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":10.0,"MKT_VALUE":1000.0}'
```

In [ ]:
# Add another position for x001, and a new position for x002
positionWriter.write_position("x001","MSFT", 10)
positionWriter.write_position("x002","MSFT", 20)

In [ ]:
qw.write_quote('MSFT', 10)

Output adds...

```
b'{"OWNER":"x001","SYMBOL":"MSFT","AMOUNT":10.0,"MKT_VALUE":100.0}'
b'{"OWNER":"x002","SYMBOL":"MSFT","AMOUNT":20.0,"MKT_VALUE":200.0}'
```

In [ ]:
# What happens if we change a position? Is it an update or just an add of another value?
positionWriter.write_position("x001","AMZN", 100)
qw.write_quote('AMZN', 100)

We observe...

```
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":10.0,"MKT_VALUE":1000.0}'
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":100.0,"MKT_VALUE":10000.0}'
```

How can we treat position writes as upserts?